In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [30]:
df = pd.read_csv("modified_data.csv")
ID = df["cell_name"]
y = df["Congestion_Type"]


In [31]:
train = df.drop(['cell_name', 'Congestion_Type'], axis=1)
# train['web_browsing_total_bytes'] += train['video_total_bytes']+ train['social_ntwrking_bytes']+ train['cloud_computing_total_bytes']+train['web_security_total_bytes']+ train['gaming_total_bytes']+ train['health_total_bytes']+ train['communication_total_bytes']+ train['file_sharing_total_bytes']+ train['remote_access_total_bytes']+ train['photo_sharing_total_bytes']+ train['software_dwnld_total_bytes']+ train['marketplace_total_bytes']+ train['storage_services_total_bytes']+ train['audio_total_bytes']+ train['location_services_total_bytes']+ train['presence_total_bytes']+ train['advertisement_total_bytes']+ train['system_total_bytes']+ train['voip_total_bytes']+ train['speedtest_total_bytes']+ train['email_total_bytes']+ train['weather_total_bytes']+ train['media_total_bytes']+ train['mms_total_bytes']+ train['others_total_bytes']
train = train.drop(['date','time','day-time','subscriber_count','web_browsing_total_bytes','video_total_bytes', 'social_ntwrking_bytes', 'cloud_computing_total_bytes', 'web_security_total_bytes',
 'gaming_total_bytes', 'health_total_bytes', 'communication_total_bytes', 'file_sharing_total_bytes', 'remote_access_total_bytes',
 'photo_sharing_total_bytes', 'software_dwnld_total_bytes', 'marketplace_total_bytes', 'storage_services_total_bytes',
 'audio_total_bytes', 'location_services_total_bytes', 'presence_total_bytes', 'advertisement_total_bytes', 'system_total_bytes',
 'voip_total_bytes', 'speedtest_total_bytes', 'email_total_bytes', 'weather_total_bytes', 'media_total_bytes',
 'mms_total_bytes', 'others_total_bytes'], axis=1)
# train['par_min']+= train['par_day']*24*60 + train['par_hour']*60
train = train.drop(['4G_rat','beam_direction', 'cell_range', 'log(total_data)', 'LOG(total_data/subs_count)', 'weekend','par_year', 'par_month', 'par_day', 'par_hour', 'par_min', 'ran_vendor', 'total_data', 'tilt',
                   'ERICSSON', 'HUAWEI', 'NOKIA', 'evening', 'night', 'work', 'total_data/subs_count', 'MAJOR/MINOR'], axis=1)
train.columns

Index(['log( web_browsing_total_bytes )', 'log( video_total_bytes )',
       'log( social_ntwrking_bytes )', 'log( cloud_computing_total_bytes )',
       'log( web_security_total_bytes )', 'log( gaming_total_bytes )',
       'log( health_total_bytes )', 'log( communication_total_bytes )',
       'log( file_sharing_total_bytes )', 'log( remote_access_total_bytes )',
       'log( photo_sharing_total_bytes )', 'log( software_dwnld_total_bytes )',
       'log( marketplace_total_bytes )', 'log( storage_services_total_bytes )',
       'log( audio_total_bytes )', 'log( location_services_total_bytes )',
       'log( presence_total_bytes )', 'log( advertisement_total_bytes )',
       'log( system_total_bytes )', 'log( voip_total_bytes )',
       'log( speedtest_total_bytes )', 'log( email_total_bytes )',
       'log( weather_total_bytes )', 'log( media_total_bytes )',
       'log( mms_total_bytes )', 'log( others_total_bytes )',
       'log( subscriber_count )'],
      dtype='object')

In [32]:
train = np.sqrt(train)

In [33]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
# train["ran_vendor"] = le.fit_transform(train["ran_vendor"])

In [34]:
x_train = train[:60000]
y_train = y[:60000]
x_test = train[60000:]
y_test = y[60000:]

In [40]:
K = 10
kf = KFold(n_splits = K, random_state = 42, shuffle = True)



In [41]:
for train_index, test_index in kf.split(x_train):
    train_X, valid_X = x_train.iloc[train_index], x_train.iloc[test_index]
    train_y, valid_y = y[train_index], y[test_index]

In [42]:
xgb_params = {
              'learning_rate': 0.151,
              'max_depth':1,
              'n_estimators':3000,
              'reg_lambda':1.5,
              'min_child_weight': 8, 
              'subsample': 0.7, 
              'seed': 42
             }

In [43]:
xgb_preds = []

In [44]:
d_train = xgb.DMatrix(train_X, train_y)
d_valid = xgb.DMatrix(valid_X, valid_y)
d_test = xgb.DMatrix(x_test)
    
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, verbose_eval=50, early_stopping_rounds=100)
                        
xgb_pred = model.predict(d_test)
xgb_preds.append(list(xgb_pred))

[0]	train-rmse:1.40169	valid-rmse:1.40552
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[50]	train-rmse:1.0651	valid-rmse:1.06967
[100]	train-rmse:1.03989	valid-rmse:1.04607
Stopping. Best iteration:
[0]	train-rmse:1.40169	valid-rmse:1.40552



In [48]:
# xgb_preds